In [ ]:
import os
path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)

In [ ]:
from ggmodel_dev.graphmodel import GraphModel, concatenate_graph_specs
from ggmodel_dev.validation import score_model, plot_diagnostic, score_model_dictionnary, model_validation_pipeline, export_to_xls

from data_utils.database import select_dataset
from data_utils.utils import get_X_y_from_data, data_dict_from_df_tables
from ggmodel_dev.models.landuse.BE2 import model_dictionnary

import pandas as pd
import numpy as np

# Model developement notebook

The goal of this notebook is to develop and improve the landuse model.

Remarks:

- comment 1
- comment 2
- comment 3

## Data

In [3]:
def prepare_landuse_model_data():

    df_tables = ['faofoodbalancesheet', 'population', 'faocrops', 'faolanduse']

    df_dictionnary = {df_table: select_dataset(df_table) for df_table in df_tables}

    df_dictionnary['faofoodbalancesheet'] = df_dictionnary['faofoodbalancesheet'][~df_dictionnary['faofoodbalancesheet'].Variable.isin(['Domestic supply quantity'])]

    df_dictionnary['faofoodbalancesheet'] = df_dictionnary['faofoodbalancesheet'].drop(columns=['group']) # for now only crop is considered
    
    data_dict = data_dict_from_df_tables([df for _, df in df_dictionnary.items()])

    data_dict['CL_baseline'] = data_dict['CL'].rename("CL_baseline")  
    data_dict['IL_baseline'] = data_dict['IL'].rename("IL_baseline") 
    data_dict['FL_baseline'] = data_dict['FL'].rename("FL_baseline") 
    data_dict['R_rate'] = pd.Series(data=0, index=data_dict['CL_baseline'].index, name='R_rate')
    data_dict['CL_corr_coef'] = pd.Series(data=1.4, index=data_dict['CL_baseline'].index, name='CL_corr_coef')

    def preprocess_dict_item(key, value):

        return (
            value.fillna(0)
        )

    
    data_dict = {k: preprocess_dict_item(k, v) for k, v in data_dict.items()}
    data_dict['CYi'] = data_dict['CYi'].replace(0, np.nan)
    return data_dict

data_dict = prepare_landuse_model_data()

In [4]:
data_dict.keys()

dict_keys(['FDKCi', 'FDKGi', 'FDi', 'FEi', 'FIi', 'FLOi', 'FPi', 'KKRi', 'NFDi', 'PDi', 'RDi', 'SDi', 'SSRi', 'SVi', 'Pop', 'CYi', 'TCLDi', 'CL', 'FL', 'IL', 'TLA', 'CL_baseline', 'IL_baseline', 'FL_baseline', 'R_rate', 'CL_corr_coef'])

In [5]:
ISO_data_dict = {k: v.loc[['FRA', 'HUN', 'DEU', 'NLD', 'BEL', 'CHE', 'BRA', 'SEN'], [2016, 2017, 2018], :] for k, v in data_dict.items()}

## Model

In [6]:
scores = score_model_dictionnary(model_dictionnary, ISO_data_dict)

TCLDi_model: Done
TCLDi_partial_model: Done
FPi_model: Done
IL_FL_model: Done
BE2_partial_model: Error: No objects to concatenate
BE2_model: Done


In [7]:
scores['BE2_model']['score_by_Variable']

,r2,correlation,rmse
Variable,,,
CL,-0.429486,NaN,5.715454e+08
FL,1.000000,1.00000,0.000000e+00
IL,-7567.697515,-0.14515,5.715454e+08


In [8]:
results_df = model_validation_pipeline(model_dictionnary['BE2_model'], ISO_data_dict)

In [ ]:
export_to_xls(results_df, 'data/BE2_model.xlsx')

## Scenario

In [ ]:
from ggmodel_dev.projection import *
from ggmodel_dev.models.landuse import BE2_scenario

In [ ]:
for name, series in data_dict.items():
    series.to_csv(f'data/BE2/{name}.csv')

In [ ]:
def get_data_dict_from_folder(folder_name):
    files = os.listdir(folder_name)
    data_dict = {file.split('.')[0] : pd.read_csv(f'{folder_name}/{file}') for file in files}
    data_dict = {name: df.set_index([col for col in df.columns if col != name]).squeeze() for name, df in data_dict.items()}
        
    return data_dict

In [ ]:
ISO = 'HUN'
data_dict_filtered = {k: v.loc[ISO, 2000:] for k, v in test.items()}

list_of_scenarios = [dict(FDKGi_target=0.9, FLOi_target=0.8, CYi_target=1.1, R_rate=0),
                     dict(FDKGi_target=0.5, FLOi_target=0.5, CYi_target=1.1, R_rate=50),
                     dict(FDKGi_target=1.5, FLOi_target=1.5, CYi_target=1.1, R_rate=100)]

simulation_results = run_scenario_list(scenario_function=BE2_scenario.run_scenario,
                                       list_of_scenarios=list_of_scenarios,
                                       projection_dictionnary=BE2_scenario.projection_dict,
                                       data_dict=data_dict_filtered
                                      )

In [ ]:
simulation_results['scenario_0']['BE2'].plot()
simulation_results['scenario_1']['BE2'].plot()
simulation_results['scenario_2']['BE2'].plot()

In [ ]:
simulation_results['scenario_0']['delta_CL'].plot()
simulation_results['scenario_1']['delta_CL'].plot()
simulation_results['scenario_2']['delta_CL'].plot()